In [65]:
import pandas as pd
import math
import requests
import sys
import time
import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed

def check_url(url, retries=3, delay=5, timeout=10):
    """Function to check if a URL is valid with retry mechanism for timeouts, skipping 404 errors, and pausing every 20 attempts."""
    status_code_messages = []
    for attempt in range(1, retries + 1):
        try:
            response = requests.get(url, timeout=timeout)
            if response.status_code == 200:
                return url, status_code_messages  # Return URL and status messages list
            elif response.status_code == 404:
                return None, status_code_messages  # Skip 404 errors without printing
            else:
                if attempt > 1:
                    status_code_messages.append(f"\nAttempt {attempt}: Received status code {response.status_code} for URL '{url}'")
                time.sleep(delay)

        except requests.Timeout:
            if attempt > 1:
                status_code_messages.append(f"\nAttempt {attempt}: Timeout occurred while fetching URL '{url}'")
            time.sleep(delay)
        except requests.RequestException as e:
            if attempt > 1:
                status_code_messages.append(f"\nAttempt {attempt}: An error occurred while fetching URL '{url}': {e}")
            time.sleep(delay)
        
        # Pause every 20 attempts
        if attempt % 20 == 0:
            status_code_messages.append(f"\nPausing for 2 seconds after {attempt} attempts...")
            time.sleep(2)
    
    return None, status_code_messages  # Return None and status messages list if retries are exhausted

def filter_urls(urls, max_workers=20):
    valid_urls = []
    filtered_count = 0
    status_messages_buffer = []  # Buffer to collect status messages
    print(f"Filtering {len(urls)} Total Urls - {datetime.datetime.now().strftime('%Y-%m-%d %I:%M:%S %p')}")

    try:
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_url = {executor.submit(check_url, url): url for url in urls}
            for i, future in enumerate(as_completed(future_to_url)):
                url = future_to_url[future]
                try:
                    result, status_messages = future.result()
                    if result:
                        valid_urls.append(result)
                        # Update progress bar only on valid responses
                        progress_percent = math.ceil(100 * ((i + 1) / len(urls)))
                        sys.stdout.write(f"\rValid Urls: {len(valid_urls)} - {progress_percent}% [{'▇' * (progress_percent // 2)}{' ' * (50 - progress_percent // 2)}]")
                        sys.stdout.flush()
                    else:
                        filtered_count += 1
                        status_messages_buffer.extend(status_messages)  # Add messages to the buffer

                except TypeError as te:
                    status_messages_buffer.append(f"\nTypeError occurred while processing URL '{url}': {te}")
                    filtered_count += 1
                except Exception as exc:
                    status_messages_buffer.append(f"\nAn error occurred while processing URL '{url}': {exc}")
                    filtered_count += 1

        # Print final completion messages
        print(f"\nFinished Filtering {len(valid_urls)} valid URLs and {filtered_count} filtered URLs - {datetime.datetime.now().strftime('%Y-%m-%d %I:%M:%S %p')}")
        
        # Print status messages
        for msg in status_messages_buffer:
            print(msg)

        return valid_urls, filtered_count  # Return explicitly

    except Exception as e:
        print(f"\nAn exception occurred in filter_urls: {e}")
        return [], 0  # Return empty lists if there's an unhandled exception
        
# Market Item API Pull
def fetch_data_with_retry(url, retries=3):
    """Function to fetch data from API with retry mechanism."""
    for attempt in range(1, retries + 1):
        try:
            response = requests.get(url)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            if attempt > 1:  # Only print for attempts greater than 1
                print(f"Attempt {attempt}: RequestException occurred for URL '{url}': {e}")
            time.sleep(5)  # Wait 5 seconds before retrying
        
        # Pause every 20 attempts
        if attempt % 20 == 0:
            print(f"Pausing for 2 seconds after {attempt} attempts...")
            time.sleep(2)
    
    # Log that retries are exhausted
    print(f"Retries exhausted for URL '{url}'")
    return {}  # Return empty dictionary if retries are exhausted

def extract_itemId(url):
    return url.rsplit('/', 1)[-1]

def  milliseconds_timerange(start_date):
    d1 = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    d2 = datetime.datetime.now()  # Current date and time
    delta = abs((d2 - d1).total_seconds())  # Difference in seconds
    milliseconds = delta * 1000  # Convert seconds to milliseconds
    return round(milliseconds)  # Round to the nearest millisecond

def second_timerange(start_date):
    d1 = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    d2 = datetime.datetime.now()  # Current date and time
    delta_seconds = abs((d2 - d1).total_seconds())
    return round(delta_seconds)

# Define constants
private_key = '29309ca885bb4c9096e08ef47213962d19137c17218b4f0a826024807837d435'
xivapi_url = 'https://xivapi.com'
universalis_api = 'https://universalis.app/api/v2/history'
worldDcRegion = '/Brynhildr'
timerangeStart = '2024-06-01'
statsWithin = f'?statsWithin={milliseconds_timerange(timerangeStart)}'
entriesWithin = f'&entriesWithin={second_timerange(timerangeStart)}'
timeZone = 'America/Chicago'
ENpcBase = 'C:\\Users\\piked\\Documents\\Ffxiv Outputs\\Ffxiv Scripts\\Input Csv\\Item.csv'
GatheringBase = 'C:\\Users\\piked\\Documents\\Ffxiv Outputs\\Ffxiv Scripts\\Input Csv\\GatheringPointBase.csv'
FishingBase = 'C:\\Users\\piked\\Documents\\Ffxiv Outputs\\Ffxiv Scripts\\Input Csv\\FishingSpot.csv'

# SaintCoinach Exported ENpcBase data
item_csv = pd.read_csv(ENpcBase, skiprows=1, low_memory=False) # Csv Location
item = item_csv.iloc[1:, :].rename(columns={"#": "Item_Id"})[['Item_Id', 'Name', 'IsUntradable']]
item_tble = item[(item['Name'].notnull()) & (item['IsUntradable'] == 'False') & (item['Item_Id'] != '1')].copy()
item_tble['urls'] = universalis_api + worldDcRegion + '/' + item_tble['Item_Id'].astype(str)

# SaintCoinach Exported Gathering Points data
gath_csv = pd.read_csv('C:\\Users\\piked\\Documents\\Ffxiv Outputs\\Ffxiv Scripts\\Input Csv\\GatheringPointBase.csv', skiprows=1, low_memory=False)
gather = gath_csv.iloc[1:, :][['#','GatheringType', 'GatheringLevel'] + [col for col in gath_csv.columns if 'Item' in col]]
fish_csv = pd.read_csv('C:\\Users\\piked\\Documents\\Ffxiv Outputs\\Ffxiv Scripts\\Input Csv\\FishingSpot.csv', skiprows=1, low_memory=False)
fish = fish_csv.iloc[1:, :][['#','GatheringLevel'] + [col for col in fish_csv.columns if 'Item' in col]]
fish['GatheringType'] = 'Fishing'
dis_land_df = pd.concat([fish,gather])

# Preparing to concat gathering item columns into one
dis_land_list = []
for i in range(8):
    col_name = f'Item[{i}]'
    tble = dis_land_df[['#','GatheringType','GatheringLevel', col_name]][dis_land_df[col_name].notnull()].rename(columns={col_name: 'Item'})
    dis_land_list.append(tble)

dland_df = pd.concat(dis_land_list)[['Item','GatheringType','GatheringLevel']]
dland_df = dland_df.groupby(['Item','GatheringType'], as_index=False) \
    .agg(GatheringLevel=pd.NamedAgg(column='GatheringLevel', aggfunc='min'))

urls = item_tble['urls'].drop_duplicates().values.tolist()

# Filter the URLs
valid_urls, filtered_count = filter_urls(urls)

# Save the valid URLs to a new list
filtered_urls = valid_urls

# Extract item IDs from filtered URLs
extracted_itemIds = [extract_itemId(url) for url in filtered_urls]

# Creating chunks of item IDs
n = 100  # Number of item IDs per chunk
sep_item_list = [extracted_itemIds[x:x+n] for x in range(0, len(extracted_itemIds), n)]

# Creating CSV list
csv_list = ["/" + ",".join(map(str, chunk)) for chunk in sep_item_list]

# Initialize variables
mrkt_hist_list = []
items_list_processed = 0
data_frames = []

print(f"Starting Market Item Request - {len(extracted_itemIds)} Total Items - {len(csv_list)} Total Item Lists - {datetime.datetime.now().strftime('%Y-%m-%d %I:%M:%S %p')}")

# Processing chunks in a loop
for item_id in csv_list:
    mrkt_req_json = fetch_data_with_retry(universalis_api + worldDcRegion + item_id + statsWithin + entriesWithin)
    mrkt_req_json_itms = mrkt_req_json.get('items', {})
    items_list_processed += 1
    filtered_itemIDs = [i for i in extracted_itemIds if str(i) in mrkt_req_json_itms and mrkt_req_json_itms[str(i)]['entries']]
    for i in filtered_itemIDs:
        try:
            hist_df = pd.json_normalize(mrkt_req_json_itms[str(i)]).explode('entries').reset_index(drop=True)
            hist_df = hist_df.merge(pd.json_normalize(hist_df['entries']), left_index=True, right_index=True).drop('entries', axis=1)
            mrkt_hist_list.append(hist_df[["itemID", "regularSaleVelocity", "nqSaleVelocity", "hqSaleVelocity", "worldName", "pricePerUnit", "quantity", "hq", "buyerName", "timestamp"]])
        except KeyError:
            print(f"KeyError Entries: {mrkt_req_json}")
            continue
    print(f'Market Item Lists: {items_list_processed} - {math.ceil(100 * (items_list_processed / len(csv_list)))}%', end='\r')

print('\nCompleted - ' + datetime.datetime.now().strftime('%Y-%m-%d %I:%M:%S %p'))

# Concatenate all dataframes into one
hist_table = pd.concat(mrkt_hist_list, ignore_index=True) \
    .rename(columns={"itemID" : "Item_Id","regularSaleVelocity" : "Reg_Sale_Vel","nqSaleVelocity" : "NQ_Sale_Vel","hqSaleVelocity" : "HQ_Sale_Vel","worldName" :  "World","pricePerUnit" : "Unit_Price","quantity" : "Qty","hq" : "HQ_Ind","buyerName" : "Buyer","timestamp" : "Timestamp"})
hist_table = hist_table[hist_table['Reg_Sale_Vel'] > 10]
# Sort by Item_Id and Timestamp
hist_table.sort_values(by=["Item_Id", "Timestamp"], ascending=[True, False], inplace=True)
hist_table.reset_index(drop=True, inplace=True)

# Adding Rank (RN)
hist_table['RN'] = hist_table.groupby("Item_Id").cumcount() + 1

# Convert Timestamp to desired timezone
hist_table['Timestamp'] = pd.to_datetime(hist_table['Timestamp'], unit='s', utc=True).dt.tz_convert(timeZone).dt.strftime('%Y/%m/%d %H:%M:%S')

# Drop duplicates
hist_table.drop_duplicates(inplace=True)

# Removing Outlier Prices
percentiles = hist_table.groupby("Item_Id")["Unit_Price"].quantile([0.05, 0.95]).unstack(level=1)

# Define a fixed threshold for significant deviation
threshold = 500  # Adjust this threshold based on your definition of "significantly different"

# Function to mark outliers
def mark_outliers(row):
    item_id = row["Item_Id"]
    unit_price = row["Unit_Price"]
    p05 = percentiles.loc[item_id, 0.05]
    p95 = percentiles.loc[item_id, 0.95]
    return (unit_price <= p05 - threshold) | (unit_price >= p95 + threshold)

# Apply the outlier condition
hist_table['Outlier'] = hist_table.apply(mark_outliers, axis=1)

# Additional Fields
hist_table['Item_Id'] = hist_table['Item_Id'].astype(str)
hist_table['Total_Purchases'] = hist_table.groupby('Item_Id')['Item_Id'].transform('count')
hist_table['Avg_Unit_Price'] = hist_table.loc[hist_table['Outlier'] != True].groupby('Item_Id')['Unit_Price'].transform('mean')
hist_table['Avg_Qty'] = hist_table.loc[hist_table['Outlier'] != True].groupby('Item_Id')['Qty'].transform('mean')
hist_table['Overall_Cost'] = hist_table['Unit_Price'] * hist_table['Qty']
hist_table['Reg_Sale_Vel_Rank'] = hist_table.groupby('Item_Id')['Reg_Sale_Vel'].rank(method='dense', ascending=False)
hist_table['Total_Purchases_Rank'] = hist_table.groupby('Item_Id')['Total_Purchases'].rank(method='dense', ascending=False)
hist_table['Avg_Unit_Price_Rank'] = hist_table.groupby('Item_Id')['Avg_Unit_Price'].rank(method='dense', ascending=False)
hist_table['Composite_Score'] = (hist_table['Reg_Sale_Vel_Rank'] * 0.7)+(hist_table['Total_Purchases_Rank'] * 0.3)+(hist_table['Avg_Unit_Price_Rank'] * 0.5)

# Assuming item_tble is a DataFrame containing item details
final_df = hist_table.merge(item_tble, on='Item_Id') \
    .merge(dland_df,how='left',left_on='Name',right_on='Item')[['Item_Id', 'Name', 'GatheringType', 'GatheringLevel', 'World', 'Reg_Sale_Vel', 'NQ_Sale_Vel', 'HQ_Sale_Vel', 'Composite_Score', 'Total_Purchases', 'Avg_Unit_Price', 'RN', 'Buyer', 'Avg_Qty', 'Unit_Price', 'Qty', 'Overall_Cost', 'Outlier', 'HQ_Ind', 'Timestamp']]
final_df['GatheringType'] = final_df['GatheringType'].fillna('Not Gathered')

display(final_df)

Filtering 16039 Total Urls - 2024-07-03 11:02:55 PM
Valid Urls: 15547 - 100% [▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇]
Finished Filtering 15547 valid URLs and 492 filtered URLs - 2024-07-03 11:13:22 PM

Attempt 2: Received status code 500 for URL 'https://universalis.app/api/v2/history/Brynhildr/24520'

Attempt 3: Received status code 500 for URL 'https://universalis.app/api/v2/history/Brynhildr/24520'

Attempt 2: Received status code 500 for URL 'https://universalis.app/api/v2/history/Brynhildr/25031'

Attempt 3: Received status code 500 for URL 'https://universalis.app/api/v2/history/Brynhildr/25031'

Attempt 2: Received status code 500 for URL 'https://universalis.app/api/v2/history/Brynhildr/25030'

Attempt 3: Received status code 500 for URL 'https://universalis.app/api/v2/history/Brynhildr/25030'

Attempt 2: Received status code 500 for URL 'https://universalis.app/api/v2/history/Brynhildr/25047'

Attempt 3: Received status code 500 for URL 'https://universalis.app/api

,Item_Id,Name,GatheringType,GatheringLevel,World,Reg_Sale_Vel,NQ_Sale_Vel,HQ_Sale_Vel,Composite_Score,Total_Purchases,Avg_Unit_Price,RN,Buyer,Avg_Qty,Unit_Price,Qty,Overall_Cost,Outlier,HQ_Ind,Timestamp
0,2,Fire Shard,Harvesting,15,Brynhildr,26292.193000,26292.193000,0.0,1.5,753,76.557769,1,Adrielle Morrigan,1150.863214,120,1000,120000,False,False,2024/06/25 23:47:29
1,2,Fire Shard,Logging,30,Brynhildr,26292.193000,26292.193000,0.0,1.5,753,76.557769,1,Adrielle Morrigan,1150.863214,120,1000,120000,False,False,2024/06/25 23:47:29
2,2,Fire Shard,Mining,20,Brynhildr,26292.193000,26292.193000,0.0,1.5,753,76.557769,1,Adrielle Morrigan,1150.863214,120,1000,120000,False,False,2024/06/25 23:47:29
3,2,Fire Shard,Quarrying,20,Brynhildr,26292.193000,26292.193000,0.0,1.5,753,76.557769,1,Adrielle Morrigan,1150.863214,120,1000,120000,False,False,2024/06/25 23:47:29
4,2,Fire Shard,Harvesting,15,Brynhildr,26292.193000,26292.193000,0.0,1.5,753,76.557769,2,Adrielle Morrigan,1150.863214,92,2000,184000,False,False,2024/06/25 23:47:09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458425,44106,Rroneek Chuck,Not Gathered,NaN,Brynhildr,11.771718,11.771718,0.0,1.5,19,1250.315789,15,Eksu Plosion,20.421053,999,6,5994,False,False,2024/07/03 10:09:19
458426,44106,Rroneek Chuck,Not Gathered,NaN,Brynhildr,11.771718,11.771718,0.0,1.5,19,1250.315789,16,Eksu Plosion,20.421053,998,19,18962,False,False,2024/07/03 10:09:17
458427,44106,Rroneek Chuck,Not Gathered,NaN,Brynhildr,11.771718,11.771718,0.0,1.5,19,1250.315789,17,Eksu Plosion,20.421053,998,20,19960,False,False,2024/07/03 10:09:16
458428,44106,Rroneek Chuck,Not Gathered,NaN,Brynhildr,11.771718,11.771718,0.0,1.5,19,1250.315789,18,Eksu Plosion,20.421053,998,20,19960,False,False,2024/07/03 10:09:15


In [60]:
import pandas as pd, os

os.chdir('C:\\Users\\piked\\Documents\\Ffxiv Outputs\\Ffxiv Scripts\\Output Csv\\')

final_df.to_csv('Hi_Mrkt_Vel v2.csv')